In [8]:
import warnings
warnings.filterwarnings('ignore')

# Biblioteca para tratativa
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re 
from collections import Counter ## Para descobrir número de repetição de um valor em uma lista


# Bibliotecas para classe Normalidade
import seaborn as sns ## Para plotar gráficos 
import numpy as np ## Para calculos matemáticos
import matplotlib.pyplot as plt ## Para plot de gráficos
from sklearn.preprocessing import MinMaxScaler ## Para padronizar os dados em reescala de Max e Min 
from statsmodels.graphics.gofplots import qqplot ## Para plot de gráficos

# Bibliotecas para estatística
import scipy.stats ## Classe com funções estatísticas
from factor_analyzer.factor_analyzer import calculate_kmo ## Para calculo de KMO e MSA
import researchpy as rs ## Para realizar a correlação entre as variáveis 


'''
    About this file:
    https://miro.com/app/board/uXjVPiH_KCg=/?share_link_id=298618246717

'''


class Tratativa:
    
    
    # ---------------------------------------------------------------------------------------------------------- #
    # Função: conhecimento_base
    #
    # Descrição: Realiza a retirada de algumas informações da base passada, alocando-os em um dataframe,
    #            explicando assim cada coluna independentemente
    #
    # Parâmetros: 1) dataframe = dataframe inicial com os dados
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    def conhecimento_base(dataframe) -> pd.DataFrame:

        ''' 

            Parâmetros:
                1) Dataframe

            Retorno:
                Dataframe com métricas de tendencia central e outros detalhes 

        '''

        ## Crio dataframe passando as colunas como coluna, em uma coluna chamada "coluna" kk
        df_html = pd.DataFrame(
            columns=['column'],
            data=[
                [coluna]
                for coluna
                in dataframe.columns
            ]
        )

        ## Trago o shape (qtd linhas e qtd colunas)
        df_shape = dataframe.shape
    

    ## A partir daqui, vou criando colunas e trazendo métricas através do método describe(), most_common, Counter entre outros

        ## Pego o tipo de cada coluna
        df_html['types'] = df_html.apply(
            lambda x: dataframe.dtypes[x['column']],
            axis=1
        )

        ## Pego alguns exemplos de dados que o atributo possui (5 primeiros)
        df_html['samples'] = df_html.apply(
            lambda x: list(dataframe[x['column']][:5]),
            axis=1
        )

        ## Trago os valores que mais se repetem em cada atributo 
        df_html['most common'] = df_html.apply(
            lambda x: '{0} -> {1}% - [{2}]'.format(
                    Counter(list(dataframe[x['column']])).most_common(1)[0][0],
                    round((Counter(list(dataframe[x['column']])).most_common(1)[0][1] / df_shape[0]) * 100, 1),
                    Counter(list(dataframe[x['column']])).most_common(1)[0][1],
                ),
            axis=1
        )

        ## Pego do menor ao maior valor, caso for de tipo 'int' ou 'float'
        df_html['ranges'] = df_html.apply(
            lambda x: '{} até {}'.format(
                dataframe[x['column']].min(), dataframe[x['column']].max())
            if 'int' in str(x['types']) or 'float' in str(x['types'])
            else '',
            axis=1
        )

        ## Trago primeiro quartil, caso for de tipo 'int' ou 'float'
        df_html['q1'] = df_html.apply(
            lambda x: dataframe[x['column']].describe()[4]
            if 'int' in str(x['types']) or 'float' in str(x['types'])
            else '',
            axis=1
        )

        ## Trago segundo quartil, caso for de tipo 'int' ou 'float'
        df_html['q2'] = df_html.apply(
            lambda x: dataframe[x['column']].describe()[5]
            if 'int' in str(x['types']) or 'float' in str(x['types'])
            else '',
            axis=1
        )

        ## Trago terceiro quartil, caso for de tipo 'int' ou 'float'
        df_html['q3'] = df_html.apply(
            lambda x: dataframe[x['column']].describe()[6]
            if 'int' in str(x['types']) or 'float' in str(x['types'])
            else '',
            axis=1
        )

        ## Pego a média, caso for de tipo 'int' ou 'float'
        df_html['mean'] = df_html.apply(
            lambda x: round(dataframe[x['column']].describe()[1], 1)
            if 'int' in str(x['types']) or 'float' in str(x['types'])
            else '',
            axis=1
        )

        ## Pego o desvio padrão, caso for de tipo 'int' ou 'float'
        df_html['std'] = df_html.apply(
            lambda x: dataframe[x['column']].describe()[2]
            if 'int' in str(x['types']) or 'float' in str(x['types'])
            else '',
            axis=1
        )
        
        ## Trago quantidade de valores presentes
        df_html['count'] = df_html.apply(
            lambda x: int(dataframe[x['column']].describe()[0]),
            axis=1
        )

        ## Trago quantidade de valores ausentes
        df_html['missings'] = df_html.apply(
            lambda x: dataframe[x['column']].isnull().sum(),
            axis=1
        )

        ## Trago quantidade de valores que não se repetem
        df_html['uniques'] = df_html.apply(
            lambda x: dataframe[x['column']].nunique(),
            axis=1
        )

        return df_html

    # ---------------------------------------------------------------------------------------------------------- #
    # Função: higienizacao
    #
    # Descrição: Realiza tratativa de input dos dados, abordando desde a desceleção de colunas, reposicionamento de coluna\
    #            e tentativa de reconhecimento de colunas 'id's'
    #
    # Parâmetros: 1) dataframe = dataframe inicial com os dados
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    
    def higienizacao(df):
        
        ''' 

            Parâmetros:
                1) Dataframe

            Retorno:
                Dataframe com métricas de tendencia central e outros detalhes.

            # OBSERVAÇÃO:
                A higienização descrita foi voltada para um conjunto de dados que variam, mas que se sabe préviamente o que poderá vir\n
                podendo assim não servir para outros tipos de datasets (não excluindo a utilização das classes abaixo!)

        '''
        
        ## Caso haja valores com 'Nan', trocamos por 0 
        df.replace(np.nan, 0, inplace = True)
        
        ## Caso haja uma linha com todos valores zerados, descartamo-as
        df = df.loc[(df!=0).any(axis=1)]

        ## Caso haja coluna com todos os valores zerados, rastreio-os
        lista_colunas_zeradas = [col for col in (df.columns) if (df[col] == 0).all()]
                
        ## Deleto-os
        for i in lista_colunas_zeradas:
            df.drop(columns = {f'{i}'}, inplace = True)
            print(f"A coluna {i} foi deletada pois haviam apenas valores zerados nela!")
            
        ## Verifico se há uma coluna de Unnamed para deletar
        try:
            df.drop(columns = {'Unnamed: 0'}, inplace = True)
        except:
            pass
        
        ## Tentaremos transformar as colunas em 'float'
        for coluna in df.columns:
            try:
                
                if ('cod' in coluna.lower()) or ('mes' in coluna.lower()) or ('dia' == coluna.lower()):
                    df[coluna] = df[coluna].astype('str')
                    
                else:    
                    df[coluna] = df[coluna].astype('float')
                    
            except:
                
                df[coluna] = df[coluna].astype('str')
                
            ## Removo colunas que aparecem com planejado
            if ('plan' in coluna.lower()) or ('_plan' in coluna.lower()):
                df.drop(columns = {f'{coluna}'}, inplace = True)
                
                ## Removo colunas que aparecem com percentuais
            if ('perc' in coluna.lower()) or ('perc_' in coluna.lower()):
                df.drop(columns = {f'{coluna}'}, inplace = True)
            
            
            ## Removo colunas que aparecem com dummy
            if ('dummy' in coluna.lower()) or ('_dummy' in coluna.lower()):
                df.drop(columns = {f'{coluna}'}, inplace = True)
                
                
             ## (RECEPTIVO) Se achar coluna de NS_real, coloca como última coluna (excluo ela e coloco de novo, famosa gambs)
            if ('ns' == coluna.lower()) or ('servico' in coluna.lower()):
                coluna_list = df[coluna]
                df.drop(columns = {f'{coluna}'}, inplace = True)
                df['NS'] = coluna_list


            ## Acho o dia em um datetime por expressão irregular
            if ('data' in coluna.lower()) or ('DAT_INSUMO' == coluna.lower()) or ('dat_' in coluna.lower()) or ('dia' == coluna.lower()):
                
                try:
                    if ('t' in df[coluna].iloc[1]):
                        ## Se o formato for YYYY/mm/ddTHH:MM:SS retiro o dia da data e aloco em uma coluna
                        df[coluna] = [df[coluna].iloc[x][:10] for x in range(len(df[coluna]))]

                        ## Acho o dd no formato YYYY/mm/dd
                        dias = [re.findall(r'\d{2}$', x)[0] for x in df[coluna]]

                    else:
                        try:
                            ## Para YYYY-mm-dd
                            dias = [re.findall('-\d{2} ', x)[0][1:-1] for x in df[coluna]]

                        except:

                            try:
                                ## Para tipo dd/mm/YYYY
                                
                                dias = [re.findall('\d{2}/', x)[0] for x in df[coluna]]

                            except:
            
                                # Para tipo YYYYmmdd
                                dias = [re.findall(r'\d{2}$', x)[0] for x in df[coluna]]
                                
                except Exception as erro:
                    print(f"Não foi possível tratar a data do arquivo, vide erro: {erro}")

                
                dias = [int(x) for x in dias]
                
                df['DIAS'] = dias
                    
            
        ## Pegaremos as colunas que conseguimos transformar em 'float' ou que já são naturalmente 'int'
        df_continuo = df.select_dtypes([int, float])
        
        ## Retornaremos o dataframe transformado e o dataframe selecionado
        return df, df_continuo
    
    
    
class Normalidade:

    
    class Analisa_normalidade:
        # ---------------------------------------------------------------------------------------------------------- #
        # Função: distribuicao_grafica
        #
        # Descrição: Realiza 3 tipos de gráficos para reconhecimento da distribuição visual dos dados. É necessário\
        #            inputar apenas dados de indicadores
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        def distribuicao_grafica(df, y = None) -> None:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos

                Retorno:
                    1) Printa 3 gráficos para acompanhamento da distribuição dos dados de cada coluna

                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int
            '''

            ## Coloco os dados em ordem crescente, me baseo pela segunda coluna
            df.sort_values(by = f'{df.columns[1]}', ascending = True, inplace = True)

        ## Realiza plot da distribuição dos dados de cada coluna do dataframe        
            for i, col in enumerate(df.columns):

                try:
                    ## Cria tamanho da figura para cada bloco de exibição
                    plt.figure(i, figsize=(30, 20))
                    ## Cria quantos blocos de exibição terão um plot
                    axes = plt.subplots(1, 3)[1]
                    ## Chama gráfico de histograma e o aloca no primeiro "axes"
                    sns.histplot(y = y, x=col, data=df, ax = axes[0])
                    ## Chama gráfico de boxplot e o aloca no segundo "axes"
                    sns.boxplot(y = y, x=col, data=df, ax = axes[1])
                    ## Chama gráfico de disperção e o aloca no terceiro "axes"
                    qqplot(df[col], line='s', ax=axes[2])

                except Exception as erro:
                    print(f"A coluna {col} não pode ser graficada, segue erro: {erro}")
                    continue

        
        # ---------------------------------------------------------------------------------------------------------- #
        # Função: teste_d_agostinho
        #
        # Descrição: Utiliza da curtose e assimetria da distribuição dos dados para realizar calculo testando hipótese\
        #            nula, dizendo que os dados provém de distribuiçãi normal 
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        def teste_d_agostinho(df) -> pd.DataFrame:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos

                Retorno:
                    1) Dataset com as colunas de input como valores, com atributos de p_value_d_agostinho, teste_estatistico_d_agostinho, para cada uma delas

                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int
            '''

            p_value_col = []
            teste_estatistico = []

            ## Percorro colunas
            for coluna in df.columns:
                try:
                    ## Realizo o calculo para cada atributo
                    valor_estatistico, p_value = scipy.stats.normaltest(df[coluna])
                except Exception as erro:
                    ## Se não conseguir realizar o calculo, só aloca a coluna no dataframe
                    print(f"Não pode ser realizado o calculo para a coluna {coluna}, provavelmente por tipagem de dados, segue erro: {erro}")
                
                ## Armazeno os valores provindos do método
                p_value_col.append(np.around(p_value, 3))
                teste_estatistico.append(valor_estatistico)

            ## Crio dataframe para alocar cada valor referente à cada atributo, que estão em index
            df_variancas = pd.DataFrame(data = {'p_value_d_agostinho':p_value_col, 'atributos':df.columns, 'teste_estatistico_d_agostinho':teste_estatistico})
            df_variancas = df_variancas.set_index('atributos') 

            return df_variancas

        # ---------------------------------------------------------------------------------------------------------- #
        # Função: teste_shapiro_wilks
        #
        # Descrição: Realiza o teste de cada coluna para/com hipótese nula de distribuição normal\
        #            retornando um dataframe com o teste estatístico e p-value
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        @staticmethod
        def teste_shapiro_wilks(df) -> pd.DataFrame:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos

                Retorno:
                    1) Dataset com as colunas de input como valores, com atributos de p_value_shapiro_wilks, teste_estatistico_shapiro_wilks,\n
                        além de calculo de curtose e assimetria da distribuição para cada uma delas
                
                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int

            '''

            ## Crio DataFrame com index no valor de cada coluna de df, com o valor da curtose e assimetra de cada coluna
            distribuicao = pd.DataFrame({
                         'Curtose': df.kurtosis(),
                         'Assimetria': df.skew()
                                })

            lista_p_value = []
            teste_estatistico = []

            for coluna in df.columns:
                try:
                    ## Realizo o calculo de shapior para cada coluna
                    valor_estatistico, p_value = scipy.stats.shapiro(df[coluna])

                except Exception as erro:
                    print(f"Não pode ser realizado o calculo para a coluna {coluna}, provavelmente por tipagem de dados, segue erro: {erro}")
                    continue

                ## Aloco em lista, cada valor
                lista_p_value.append(np.around(p_value, 3))
                teste_estatistico.append(valor_estatistico)
                
            ## Coloco no dataframe criado anteriormente
            distribuicao['p_value_shapiro_wilks'] = lista_p_value
            distribuicao['teste_estatistico_shapiro_wilks'] = teste_estatistico
            
            return distribuicao


        # ---------------------------------------------------------------------------------------------------------- #
        # Função: testes_normalidade
        #
        # Descrição: Chama os testes que verificam normalidade através de testes de hipótese e calculo da distribuição
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        def testes_normalidade(df) -> pd.DataFrame:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos

                Retorno:
                    1) Dataset com as colunas de input como valores, com atributos de p_value_shapiro_wilks, teste_estatistico_shapiro_wilks\n
                        p_value_d_agostinho, teste_estatistico_d_agostinho para cada uma delas, além de calculo de curtose e assimetria da distribuição

                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int

            '''
            
            ## Chamo as funções que realizam os testes
            shapiro_wilks = Normalidade.Analisa_normalidade.teste_shapiro_wilks(df)
            d_agostinho = Normalidade.Analisa_normalidade.teste_d_agostinho(df)
                 
            try:
                ## Realizo o join pelo index das tabelas (que são as colunas)
                df_all = pd.merge(
                    left = shapiro_wilks,
                    right = d_agostinho,
                    how = 'left',
                    on = shapiro_wilks.index
                )
                
                ## Renomeio primeira coluna, que vem como default 'key_o'
                df_all.rename(columns = {'key_0' : 'atributos'}, inplace = True)
                
                
            except Exception as erro:
                print(f"Não foi possível realizar a junção dos testes, vide erro {erro}")

            return df_all 
    
    

    class Reajusta_dados:
    
        # ---------------------------------------------------------------------------------------------------------- #
        # Função: padronizacao_z_score
        #
        # Descrição: Realiza a padronização por z_score que trata cada valor pela média e pela variança do conjutno\
        #            que o mesmo pertence
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        @staticmethod
        def padronizacao_z_score(df) -> pd.DataFrame:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos 

                Retorno:
                    1) Dataset com os valores transformados a partir da formula de Z_Score

                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int
            '''

            ## Crio dataframe
            score_z = pd.DataFrame()
            for col in df.columns:
                try:
                    ## Crio coluna com o nome da coluna original e coloco o sufixo '_SCORE_Z', alocando o valor do calculo
                    score_z[f'{col}' + '_score_z'] = scipy.stats.zscore(df[col], nan_policy='omit')
                except:
                    ## Se não conseguir realizar o calculo, só aloca a coluna no dataframe
                    score_z[f'{col}'] = list(df[f'{col}'])

            return score_z



        # ---------------------------------------------------------------------------------------------------------- #
        # Função: normalizacao_1_0
        #
        # Descrição: Realiza a normalização do dado baseado no seu conjunto, utilizando o máximo e o mínimo do conjunto\
        #            que o mesmo pertence
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        @staticmethod
        def normalizacao_1_0(df) -> pd.DataFrame:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos 

                Retorno:
                    1) Dataset com os valores transformados com relação ao seus Máximos e Mínimos

                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int
            '''


            ## Explicito o método
            escala = MinMaxScaler()
            ## Utilizo a transformação para os valores de df
            array_normalizado = escala.fit_transform(df)

            ## Crio dataframe com esses valores nomralizados
            df_normalizado = pd.DataFrame(array_normalizado, 
                                      columns=df.columns,
                                      index=df.index)
            
            [df_normalizado.rename(columns = {col : f'{col}_maxmin'}, inplace = True) for col in df_normalizado.columns]

            return df_normalizado


        # ---------------------------------------------------------------------------------------------------------- #
        # Função: normalizacao_log
        #
        # Descrição: Realiza a transformação do valor em log inverso na base 'e', ou seja log(e(exp(x))) , que é chamado de logarítimo natural
        #
        # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
        #
        # Quem procurar: Otávio Augusto Iasbeck
        # ---------------------------------------------------------------------------------------------------------- #
        def transformacao_logaritimica(df) -> pd.DataFrame:

            '''
                Parâmetro: 
                    1) df: Dataset com atributos 

                Retorno:
                    1) Dataset com os valores transformados para logarítimo
                
                #### Observações:
                    É necessário que as colunas estejam em tipagem numérica como Float ou Int

            '''

            df_sqrt = pd.DataFrame()
            for i in df.columns:
                df_sqrt[f'{i}_log'] = np.log(df[i])

            df_sqrt.dropna(axis = 1, inplace = True)
            return df_sqrt




class Varianca:

    # ---------------------------------------------------------------------------------------------------------- #
    # Função: teste_levene
    #
    # Descrição: Realiza teste de levene para todas combinações de colunas\
    #            alocando o valor com p_valor > nível de significância em um dicionário
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    @staticmethod
    def teste_levene(df) -> pd.DataFrame: 
        
        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                1) Dataset com as colunas inputadas como valores de uma coluna, possuindo atributos de p_value_Levene e teste_estatistico_levene do teste de Levene
            
            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''

        relacoes = []
        p_value_col = []
        teste_estatistico = []

        ## Percorro as colunas 2x
        for coluna_x in df.columns:
            for coluna_y in df.columns:
                
                ## Para não ocasionar em colunas iguais
                if coluna_x == coluna_y:
                    continue
                    
                ## Colocamos o parâmetro de center = 'mean' pois na formula podemos utilizar como variação dentro do calculo a mediana ou o média\
                ## e nos testes foi-se descoberto que a média trazia melhor adequação quanto à conjuntos não normais de distribuição
                valor_estatistico, p_value = scipy.stats.levene(df[coluna_x], df[coluna_y])

                
                relacoes.append(f'{coluna_x} & {coluna_y}')
                p_value_col.append(np.around(p_value, 4))
                teste_estatistico.append(valor_estatistico)
                
        df_variancas = pd.DataFrame(data = {'p_value_Levene':p_value_col, 'atributos':relacoes, 'teste_estatistico_levene':teste_estatistico})
        df_variancas = df_variancas.set_index('atributos') 
            
        #df_variancas.drop_duplicates(subset = ['p_value_Levene', 'teste_estatistico_levene'], inplace = True)
            
        return df_variancas
    
    
    # ---------------------------------------------------------------------------------------------------------- #
    # Função: teste_bartlett
    #
    # Descrição: Realiza teste de levene para todas combinações de colunas\
    #            alocando o valor com p_valor > nível de significância em um dicionário
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    @staticmethod
    def teste_bartlett(df) -> pd.DataFrame: 

        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                1) Dataset com as colunas inputadas como valores de uma coluna, possuindo atributos de p_value_Bartlett e teste_estatistico_bartlett do teste de Bartlett

            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''

        relacoes = []
        p_value_col = []
        teste_estatistico = []
        
        ## Percorro as colunas 2x
        for coluna_x in df.columns:
            for coluna_y in df.columns:
                
                ## Para não ocasionar em colunas iguais
                if coluna_x == coluna_y:
                    continue
                
                ## Realizo o calculo e trago somente o p_value
                valor_estatistico, p_value = scipy.stats.bartlett(df[coluna_x], df[coluna_y])
                
                relacoes.append(f'{coluna_x} & {coluna_y}')
                p_value_col.append(np.around(p_value, 4))
                teste_estatistico.append(valor_estatistico)
                
        df_variancas = pd.DataFrame(data = {'p_value_Bartlett':p_value_col, 'atributos':relacoes, 'teste_estatistico_bartlett':teste_estatistico})
        df_variancas = df_variancas.set_index('atributos')     
        
        #df_variancas.drop_duplicates(subset = ['p_value_Bartlett', 'teste_estatistico_bartlett'], inplace = True)
        
        return df_variancas
    
    
    # ---------------------------------------------------------------------------------------------------------- #
    # Função: testes_variancas
    #
    # Descrição: Realiza a junção dos valores de varianças que foram obtidos realizando a combinação entre os atributos\
    #            posteriormente realizado join com index
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    def testes_variancas(df) -> pd.DataFrame:

        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                1) Dataset com as colunas inputadas como valores de uma coluna, possuindo atributos de p_value e r_statics do teste de Levene e Bartlett\
            
            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''

        ## Chamo as funções de testes 
        levene = Varianca.teste_levene(df)
        bartlett = Varianca.teste_bartlett(df)
        
        try:
            ## Realizo o join pelo index
            testes_variancas = pd.merge(
                left = levene,
                right = bartlett,
                how = 'left',
                on = levene.index
            )
                
            ## Renomeio primeira coluna, que vem como default 'key_o'
            testes_variancas.rename(columns = {'key_0' : 'atributos'}, inplace = True)
        
        except Exception as erro:
            print(f"Não foi possível realizar a junção dos testes, vide erro {erro}")
                
        return testes_variancas


class Correlacao():

    # ---------------------------------------------------------------------------------------------------------- #
    # Função: kmo_msa
    #
    # Descrição: Realiza o calculo de MSA que visa buscar a signficância explicativa de uma variável\
    #            enquanto o KMO trás um valor de significância geral dos atributos, visando a análise\
    #            de uma aplicação fatorial
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    def kmo_msa(df):

        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                1) Dataset com as colunas inputadas como valores de uma coluna, possuindo um atributo com valor de MSA para cada coluna\n
                2) Valor de KMO 

            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''
        
        try:
            
            ## Crio dataframe com os valores de MSA para cada coluna, que estão como index
            kmo_aplicado = pd.DataFrame(
                            list(calculate_kmo(df)[0]),
                            index=list(df.columns),
                            columns = ['MSA']
                           )
            
        except Exception as erro:
            print(f"Não foi possível aplicar o modelo, vide erro: {erro}")
        
        return calculate_kmo(df)[1], kmo_aplicado
        
        
    # ---------------------------------------------------------------------------------------------------------- #
    # Função: teste_sperman
    #
    # Descrição: Realiza o teste de Spearman de 2 formas, a primeira ele realiza, aloca em um dataframe\
    #            junto às colunas combinatórias como index e colunas informando os valores de retorno do teste\
    #            Também realiza o teste através de outro método, alocando os coeficientes positivos e negativos\
    #            em dicionários
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    def teste_sperman(df):
        
        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                1) Dataset com as colunas inputadas como valores de uma coluna, possuindo atributos de p_value e r_statics do teste de Sperman\n
                2) Dicionários com valores de correlações positivas (calculado com módulo diferente do primeiro retorno)\n
                3) Dicionário com valores de correlações negativas (calculado com módulo diferente do primeiro retorno)

            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''

        atributos_correlacionados_negativos = []
        atributos_correlacionados_positivos = []

        ## Faço o primeiro teste, através do módulo 'correlation', que nos retorna um dataframe de correlações combinatórias
        corr = rs.correlation.corr_pair(df, method= 'spearman')
        corr.rename(columns = {'N': 'nmr_valores_spearman',
                              'p-value':'p_value_spearman',
                              'r value':'r_value_spearman'}, inplace = True)
        
        ## Tratativa para trazer todas as colunas, menos a 'NS_REAL'
        #colunas_a[:] = (x for x in df.columns if x != "NS_REAL")
        
        ## Excluo a última coluna (baseando-se que a última coluna é a classe, pela tratativa inicial dos dados)
        atributos = df.drop(columns = {f'{str(df.columns[len(df.columns)-1:][0])}'})
        atributos = df
        
        ## Percorro as colunas do df 2x
        for coluna_x in atributos.columns:
            for coluna_y in atributos.columns:
                ## Aplico o método de spearman pelo módulo 'scipy.stats'
                correlacao_value, valor_p = scipy.stats.spearmanr(atributos[coluna_x], atributos[coluna_y])
                ## Se forem a mesma coluna, pula
                if ((str(coluna_y) == str(coluna_x)) == False):
                    ## Se obtiver um correlação significativa
                    if (correlacao_value > 0.6) or (correlacao_value < -0.6):
                        ## Se for uma correlação negativa
                        if correlacao_value < 0:
                            atributos_correlacionados_negativos.append(f"{coluna_x} : {coluna_y} -> {correlacao_value} com valor-p {valor_p}")
                        else:
                            atributos_correlacionados_positivos.append(f"{coluna_x} : {coluna_y} -> {correlacao_value} com valor-p {valor_p}")
        
        return corr, atributos_correlacionados_positivos, atributos_correlacionados_negativos
    
    
    # ---------------------------------------------------------------------------------------------------------- #
    # Função: teste_pearson
    #
    # Descrição: Realiza o teste de Pearson de 2 formas, a primeira ele realiza, aloca em um dataframe\
    #            junto às colunas combinatórias como index e colunas informando os valores de retorno do teste\
    #            Também realiza o teste através de outro método, alocando os coeficientes positivos e negativos\
    #            em dicionários
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    def teste_pearson(df):

        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                1) Dataset com as colunas inputadas como valores de uma coluna, possuindo atributos de p_value e r_statics do teste de Pearson\n
                2) Dicionários com valores de correlações positivas (calculado com módulo diferente do primeiro retorno)\n
                3) Dicionário com valores de correlações negativas (calculado com módulo diferente do primeiro retorno)

            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''

        atributos_correlacionados_negativos = []
        atributos_correlacionados_positivos = []

        ## Faço o primeiro teste, através do módulo 'correlation', que nos retorna um dataframe de correlações combinatórias
        corr = rs.correlation.corr_pair(df, method= 'pearson')
        corr.rename(columns = {'N': 'nmr_valores_pearson',
                              'p-value':'p_value_pearson',
                              'r value':'r_value_pearson'}, inplace = True)
        
        ## Excluo a última coluna (baseando-se que a última coluna é a classe, pela tratativa inicial dos dados)
        atributos = df.drop(columns = {f'{str(df.columns[len(df.columns)-1:][0])}'})
        atributos = df
        
        ## Percorro as colunas do df 2x
        for coluna_x in atributos.columns:
            for coluna_y in atributos.columns:
                ## Aplico o método de spearman pelo módulo 'scipy.stats'
                correlacao_value, valor_p = scipy.stats.pearsonr(atributos[coluna_x], atributos[coluna_y])
                ## Se forem a mesma coluna, pula
                if ((str(coluna_y) == str(coluna_x)) == False):
                    ## Se obtiver um correlação significativa
                    if (correlacao_value > 0.6) or (correlacao_value < -0.6):
                        ## Se for uma correlação negativa
                        if correlacao_value < 0:
                            atributos_correlacionados_negativos.append(f"{coluna_x} : {coluna_y} -> {correlacao_value} com valor-p {valor_p}")
                        else:
                            atributos_correlacionados_positivos.append(f"{coluna_x} : {coluna_y} -> {correlacao_value} com valor-p {valor_p}")
        
        return corr, atributos_correlacionados_positivos, atributos_correlacionados_negativos
    
    
    # ---------------------------------------------------------------------------------------------------------- #
    # Função: testes_correlacoes
    #
    # Descrição: Realiza a junção dos valores de correlação que foram obtidos realizando a combinação entre os atributos\
    #            posteriormente realizado join com index
    #
    # Parâmetros: 1) dataframe = dataframe com dados formatados pela tipagem e por indicadores
    #
    # Quem procurar: Otávio Augusto Iasbeck
    # ---------------------------------------------------------------------------------------------------------- #
    def testes_correlacoes(df) -> pd.DataFrame:

        '''
            Parâmetro: 
                1) df: Dataset com atributos de tipagem numérica (float, int)

            Retorno:
                Dataset com as colunas inputadas como valores de uma coluna, possuindo atributos de p_value e r_statics do teste de Sperman e Pearson
            
            #### Observações:
                É necessário que as colunas estejam em tipagem numérica como Float ou Int
        '''

        ## Chamo as funções de testes e pego o primeiro return de cada uma 
        sperman = Correlacao.teste_sperman(df)[0]
        pearson = Correlacao.teste_pearson(df)[0]
        
        try:
            ## Realizo o join pelo index das tabelas
            testes_correlacoes = pd.merge(
                left = sperman,
                right = pearson,
                how = 'left',
                on = sperman.index
            )
                
        except Exception as erro:
            print(f"Não foi possível realizar a junção dos testes, vide erro {erro}")
                
        ## Renomeio primeira coluna, que vem como default 'key_o'
        testes_correlacoes.rename(columns = {'key_0' : 'atributos'}, inplace = True)
                
        return testes_correlacoes
    


In [ ]:
Tratativa.conhecimento_base(df)

In [ ]:
df_new, df_continuo = Tratativa.higienizacao(df)

### OBS: Daqui em diante, foram feitos alguns códigos para auxílio na compilção desses testes e facilidade na interpretação baseando-se em regras de aceitação , porém cada função pode ser chamado individualmente!

## Verifica normalidade

In [ ]:
Normalidade.Analisa_normalidade.distribuicao_grafica(df)

### Criando regras

In [44]:
significancia_norm = 0.05
porcentagem_base_aceitavel = 30

In [45]:
def regras_normalidade(significancia_norm, porcentagem_base_aceitavel, df):
    
    df_analise_normalidade = Normalidade.Analisa_normalidade.testes_normalidade(df)
    ## Verificando por p_value
    normalidade_aceitavel = df_analise_normalidade.loc[
                                                        (df_analise_normalidade['p_value_shapiro_wilks'] > significancia_norm) |\
                                                        (df_analise_normalidade['p_value_d_agostinho'] > significancia_norm)
                                                      ]
    
    if len(normalidade_aceitavel) >= round((len(df_analise_normalidade) * (porcentagem_base_aceitavel/100))):
        print(f"A base de dados possui {len(normalidade_aceitavel)} atributos ({list(normalidade_aceitavel['atributos'])}) que seguem uma distribuição normal\
 Sendo assim, igual ou maior que {porcentagem_base_aceitavel}% da base de dados")
        aceitavel = 's'
    
    else:
        print(f"A base de dados possui {len(normalidade_aceitavel)} atributos ({list(normalidade_aceitavel['atributos'])}) que seguem uma distribuição normal\n\
Não conseguindo atingir a quantidade aceitavel passada como parâmetro!({porcentagem_base_aceitavel}%).\n\
Sugere-se um pré-processamento nos dados para prosseguir com a análise de correlação")
        aceitavel = 'n'
        
        return aceitavel

In [ ]:
aceitavel = regras_normalidade(significancia_norm, porcentagem_base_aceitavel, df_continuo)

In [ ]:
if aceitavel == 'n':
    print("Visto que a porcentagem de aceitação não foi atendida, será aplicado remodelagens de dados como normalização e padronização\n\
    afim de conseguir um percentual aceitável de normalidade")
    print("\n ----- Aplicando Padronização Z_Score--------\n")
    df_z_score = Normalidade.Reajusta_dados.padronizacao_z_score(df_continuo)
    
    aceitavel = regras_normalidade(significancia_norm, porcentagem_base_aceitavel, df_z_score)
    
if aceitavel == 'n':
    print("\n ----- Aplicando Normalização MaxMin --------\n")
    df_max_min = Normalidade.Reajusta_dados.normalizacao_1_0(df_continuo)

    aceitavel = regras_normalidade(significancia_norm, porcentagem_base_aceitavel, df_max_min)

if aceitavel == 'n':
    print("\n ----- Aplicando Transformacao Logarítimica --------\n")
    df_log = Normalidade.Reajusta_dados.transformacao_logaritimica(df_continuo)

    aceitavel = regras_normalidade(significancia_norm, porcentagem_base_aceitavel, df_log)

## Verifica variança semelhante

### Criando regras 

In [20]:
significancia_vari = 0.05

In [23]:
def regras_variancias(significancia_vari, df):
    
    df_analise_varianca = Varianca.testes_variancas(df)
        
    ## Verificando por p_value
    varianca_aceitavel = df_analise_varianca.loc[
                                                        (df_analise_varianca['p_value_Levene'] >= significancia_vari) |
                                                        (df_analise_varianca['p_value_Bartlett'] >= significancia_vari)
                                                      ]
    varianca_aceitavel.drop_duplicates(subset = ['p_value_Levene', 'teste_estatistico_levene', 'p_value_Bartlett', 'teste_estatistico_bartlett'], inplace = True)
    
    if len(varianca_aceitavel) > 0:
        for i in range(len(varianca_aceitavel)):
            print("------------")
            print(f"Os atributos que possuem a mesma variança foram: {varianca_aceitavel['atributos'].iloc[i]}\nCom correlação de {round(varianca_aceitavel['teste_estatistico_levene'].iloc[i], 3)} para teste de Levene e {round(varianca_aceitavel['teste_estatistico_bartlett'].iloc[i], 2)} para Bartlett") 
    else:
        print(f"Não foram encontradas varianças entre as variáveis que passem no teste de significancia passada como parâmetro!")
    return df_analise_varianca, varianca_aceitavel

In [ ]:
df_analise_varianca_, varianca_aceitavel_ = regras_variancias(significancia_vari, df_continuo)

## Verifica correlações

### Criando regras 

In [122]:
significancia_sperman = 0.03
significancia_pearson = 0.03
msa_aceitavel = 0
correlacao_positiva = 0.5
correlacao_negativa = -0.5
porcentagem_base_aceitavel = 30

In [123]:
def regras_correlacoes(significancia_sperman, significancia_pearson, msa_aceitavel, correlacao_positiva, correlacao_negativa, porcentagem_base_aceitavel, df):
    
    #df_analise_correlacao.set_index('atributos', inplace = True)
    
    df_analise_correlacao = Correlacao.testes_correlacoes(df)
    
    atr = [df_analise_correlacao.atributos]
    df_analise_correlacao.drop(columns = {'atributos'}, inplace = True)
    df_analise_correlacao = df_analise_correlacao.astype('float')
    df_analise_correlacao['atributos'] = atr[0]
    
    
    ## Verificando por p_value
    correlacao_aceitavel = df_analise_correlacao.loc[
        
                                        ((df_analise_correlacao['p_value_spearman'] > significancia_sperman) |
                                        (df_analise_correlacao['p_value_pearson'] > significancia_pearson)) 
                                        &
                                        ((df_analise_correlacao['r_value_spearman'] > correlacao_positiva) |
                                        (df_analise_correlacao['r_value_spearman'] <= correlacao_negativa))
        
                                                  ]
    
    kmo, correlacao_aceitavel_kmo = Correlacao.kmo_msa(df_ordinais)
    
    
    correlacao_aceitavel_kmo = correlacao_aceitavel_kmo.loc[
                                    (correlacao_aceitavel_kmo['MSA'] > msa_aceitavel)
                                                ].sort_values(by = 'MSA', ascending = False)
    
    
    if len(correlacao_aceitavel) != 0:
        print("Temos as seguintes correlações:")
        
        for row in range(len(correlacao_aceitavel)):    
            ## Apenas ordeno os dados do maior para o menor
            correlacao_aceitavel.sort_values(by = 'r_value_spearman', ascending = False, inplace = True)
            
            informa = f"{correlacao_aceitavel['atributos'].iloc[row]} com correlação de {correlacao_aceitavel['r_value_spearman'].iloc[row]}\n"
            print(informa)
            
    else:
        print("Não houve correlação que tenha passado no limites de aceitação!\n")
        
        
    for i in range(len(correlacao_aceitavel_kmo)):
        if correlacao_aceitavel_kmo['MSA'].iloc[i] > 0:
            
            print(f"O atributo {correlacao_aceitavel_kmo.index[i]} possui {round((correlacao_aceitavel_kmo['MSA'].iloc[i] * 100))}% de impacto na importância entre os indicadores para explicação dos outros indicadores da operação")
            #print(f'KMO = {kmo}')
        
    return correlacao_aceitavel_kmo, correlacao_aceitavel

In [ ]:
correlacao_aceitavel_kmo, correlacao_aceitavel = regras_correlacoes(significancia_sperman, significancia_pearson, msa_aceitavel, correlacao_positiva, correlacao_negativa, porcentagem_base_aceitavel, df_ordinais)